# Importing Resnet model and exploring

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet18,ResNet18_Weights
from torch.utils.data import DataLoader,random_split


mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

train_tf = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

test_tf = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_data= datasets.CIFAR10(root="./data", train=True, transform=train_tf, download=True)
test_data= datasets.CIFAR10(root="./data", train=False, transform=test_tf, download=True)

train_set, val_set = random_split(train_data, [40000,10000])

train_loader= DataLoader(train_set, batch_size=128, shuffle=True)
val_loader= DataLoader(val_set, batch_size=128, shuffle=False)
test_loader= DataLoader(test_data, batch_size=128, shuffle=False)

Files already downloaded and verified


/home/aayam/micromamba/envs/mnist/lib/python3.11/site-packages/torchvision/datasets/cifar.py:83: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  entry = pickle.load(f, encoding="latin1")


Files already downloaded and verified


In [12]:
device= device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

weights= ResNet18_Weights.DEFAULT
model= resnet18(weights=weights).to(device)

cuda


In [17]:
num_features= model.fc.in_features
model.fc= nn.Linear(num_features, 10)
model.fc= model.fc.to(device)

In [21]:
for param in model.parameters():
    param.requires_grad= False


for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [22]:
criterion= nn.CrossEntropyLoss()

optimizer= torch.optim.AdamW(model.parameters(), lr=0.001)

best_loss= float("inf")

num_epochs=20

for epoch in range(num_epochs):
    model.train()
    running_loss=0 

    for batchidx, (image,label) in enumerate(train_loader):
        
        image, label = image.to(device), label.to(device)

        output= model(image)
        loss= criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (batchidx+1) ==100:

         print(f"epoch {epoch+1}/{num_epochs} | batch {batchidx+1}/{len(train_loader)}| loss:{loss.item()}")

    
    avg_loss= running_loss/len(train_loader)
    print(f"epoch {epoch+1}/{num_epochs} | avg loss:{avg_loss}")

    model.eval()
    with torch.no_grad():
       running_val_loss=0

       for image,label in val_loader:
          
          image, label = image.to(device), label.to(device)
          
          output= model(image)
          loss= criterion(output,label)

          running_val_loss += loss.item()
        
       avg_val_loss= running_val_loss/len(val_loader)
       print(f"epoch {epoch+1}/{num_epochs} avg val loss: {avg_val_loss}")

       if avg_val_loss < best_loss:
          best_loss= avg_val_loss
          torch.save(model.state_dict(),"good_model.pth")


epoch 1/20 | batch 100/313| loss:0.38945528864860535
epoch 1/20 | avg loss:0.44965928078840334
epoch 1/20 avg val loss: 0.3979668489928487
epoch 2/20 | batch 100/313| loss:0.27123787999153137
epoch 2/20 | avg loss:0.25828384208126953
epoch 2/20 avg val loss: 0.27894840625267997
epoch 3/20 | batch 100/313| loss:0.21290139853954315
epoch 3/20 | avg loss:0.1879881162184496
epoch 3/20 avg val loss: 0.26673758680684656
epoch 4/20 | batch 100/313| loss:0.15780583024024963
epoch 4/20 | avg loss:0.1484073917158305
epoch 4/20 avg val loss: 0.2607922535153884
epoch 5/20 | batch 100/313| loss:0.1216164380311966
epoch 5/20 | avg loss:0.12316297117275551
epoch 5/20 avg val loss: 0.29182459027329577
epoch 6/20 | batch 100/313| loss:0.060841236263513565
epoch 6/20 | avg loss:0.10311736065311172
epoch 6/20 avg val loss: 0.26377509666394583
epoch 7/20 | batch 100/313| loss:0.12543027102947235
epoch 7/20 | avg loss:0.08797077734035234
epoch 7/20 avg val loss: 0.28270052396987055
epoch 8/20 | batch 100/3

In [23]:
model.load_state_dict(torch.load("good_model.pth", map_location=device))
model.to(device)

total=0
correct=0

model.eval()
with torch.no_grad():
    for images,labels in test_loader:
        images= images.to(device)
        labels= labels.to(device)

        output=model(images)
        _,predicted= torch.max(output,1)

        total+= images.size(0)
        correct+= (predicted == labels).sum().item()

    accuracy= 100 * correct/total
    print(accuracy)


/tmp/ipykernel_10538/2324255352.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("good_model.pth", map_location=device))


91.27
